In [1]:
#Data cleaning

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
df = pd.read_csv(r"C:\Users\karunya\Documents\Guvi projects\Agricultural Analysis Project\Agri_explorer.csv")

# Ensure column names are correct
df.columns = df.columns.str.strip().str.lower()  # Standardize column names

df.columns = (
    df.columns.str.lower()  # Convert to lowercase
    .str.replace(r"[()]", "", regex=True)  # Remove parentheses
    .str.replace(r"\s+", "_", regex=True)  # Replace spaces with underscores
)

In [ ]:
print(df.columns)

In [3]:
# Additional renaming for crop-related columns
df.columns = df.columns.str.replace("_area_1000_ha", "_area", regex=True)
df.columns = df.columns.str.replace("_production_1000_tons", "_production", regex=True)
df.columns = df.columns.str.replace("_yield_kg_per_ha", "_yield", regex=True)

In [5]:
print(df.columns)

Index(['dist_code', 'year', 'state_code', 'state_name', 'dist_name',
       'rice_area', 'rice_production', 'rice_yield', 'wheat_area',
       'wheat_production', 'wheat_yield', 'kharif_sorghum_area',
       'kharif_sorghum_production', 'kharif_sorghum_yield',
       'rabi_sorghum_area', 'rabi_sorghum_production', 'rabi_sorghum_yield',
       'sorghum_area', 'sorghum_production', 'sorghum_yield',
       'pearl_millet_area', 'pearl_millet_production', 'pearl_millet_yield',
       'maize_area', 'maize_production', 'maize_yield', 'finger_millet_area',
       'finger_millet_production', 'finger_millet_yield', 'barley_area',
       'barley_production', 'barley_yield', 'chickpea_area',
       'chickpea_production', 'chickpea_yield', 'pigeonpea_area',
       'pigeonpea_production', 'pigeonpea_yield', 'minor_pulses_area',
       'minor_pulses_production', 'minor_pulses_yield', 'groundnut_area',
       'groundnut_production', 'groundnut_yield', 'sesamum_area',
       'sesamum_production', 'sesa

In [15]:
import numpy as np

# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Identify yield columns (columns that contain "yield" in their name)
yield_columns = [col for col in df.columns if "yield" in col]

# Replace zero values in yield columns with NaN
df[yield_columns] = df[yield_columns].replace(0, np.nan)

# Save the cleaned dataset
df.to_csv("Cleaned_Agri_Data.csv", index=False)

print("Cleaned CSV file saved as Cleaned_Agri_Data.csv")


Cleaned CSV file saved as Cleaned_Agri_Data.csv


In [16]:
# List of columns to drop (since they lack production and yield data)
cols_to_drop = [
    "fruits_area",
    "vegetables_area",
    "fruits_and_vegetables_area",
    "potatoes_area",
    "onion_area",
    "fodder_area"
]

# Drop them from the DataFrame
df = df.drop(columns=cols_to_drop, errors="ignore") 
df.columns

Index(['dist_code', 'year', 'state_code', 'state_name', 'dist_name',
       'rice_area', 'rice_production', 'rice_yield', 'wheat_area',
       'wheat_production', 'wheat_yield', 'kharif_sorghum_area',
       'kharif_sorghum_production', 'kharif_sorghum_yield',
       'rabi_sorghum_area', 'rabi_sorghum_production', 'rabi_sorghum_yield',
       'sorghum_area', 'sorghum_production', 'sorghum_yield',
       'pearl_millet_area', 'pearl_millet_production', 'pearl_millet_yield',
       'maize_area', 'maize_production', 'maize_yield', 'finger_millet_area',
       'finger_millet_production', 'finger_millet_yield', 'barley_area',
       'barley_production', 'barley_yield', 'chickpea_area',
       'chickpea_production', 'chickpea_yield', 'pigeonpea_area',
       'pigeonpea_production', 'pigeonpea_yield', 'minor_pulses_area',
       'minor_pulses_production', 'minor_pulses_yield', 'groundnut_area',
       'groundnut_production', 'groundnut_yield', 'sesamum_area',
       'sesamum_production', 'sesa

In [17]:
# Define the common columns that should be retained in every crop DataFrame
common_columns = {"dist_code", "year", "state_code", "state_name", "dist_name"}

# Identify unique crop names dynamically (remove common columns)
crop_names = set(col.split("_crop_")[0] for col in df.columns if "_crop_" in col)

# Dictionary to store DataFrames for each crop
crop_dfs = {}

for crop in crop_names:
    # Select common columns + the current crop's specific columns
    crop_columns = list(common_columns) + [f"{crop}_crop_area", f"{crop}_crop_production", f"{crop}_crop_yield"]

    # Ensure the required columns exist in the DataFrame before filtering
    existing_columns = [col for col in crop_columns if col in df.columns]

    # Create a new DataFrame for the current crop
    crop_dfs[crop] = df[existing_columns].copy()

# Now crop_dfs dictionary contains separate DataFrames for each crop

for crop, crop_df in crop_dfs.items():
    zero_mask = (crop_df[f"{crop}_crop_area"] == 0) & \
                (crop_df[f"{crop}_crop_production"] == 0) & \
                (crop_df[f"{crop}_crop_yield"] == 0)
    
    zero_count = zero_mask.sum()
    
    print(f"Crop: {crop}, Zero Rows: {zero_count}")

In [18]:
# Save the cleaned dataset as a CSV file
df.to_csv("Agri_explorermod.csv", index=False)

In [19]:
# Select columns related to rice and wheat
rice_columns = [col for col in df.columns if "rice" in col]
wheat_columns = [col for col in df.columns if "wheat" in col]

# Create separate DataFrames for rice and wheat
df_rice = df[["state_code", "state_name", "dist_code", "dist_name", "year"] + rice_columns]
df_wheat = df[["state_code", "state_name", "dist_code", "dist_name", "year"] + wheat_columns]

# Save them as separate CSV files
df_rice.to_csv("Rice_data.csv", index=False)
df_wheat.to_csv("Wheat_data.csv", index=False)



In [20]:
# Drop rice and wheat columns
df_other = df.drop(columns=rice_columns + wheat_columns)

# Save the remaining data to a new CSV file
df_other.to_csv("Other_Crops_data.csv", index=False)



In [21]:
# Extract Unique Values
unique_districts = df[['dist_code', 'dist_name', 'state_code', 'state_name']].drop_duplicates()
unique_states = df[['state_code', 'state_name']].drop_duplicates()
unique_years = df[['year']].drop_duplicates()

In [22]:
# Extract Unique States & Save
df_states = df[['state_code', 'state_name']].drop_duplicates()
df_states.to_csv("states.csv", index=False)

In [23]:
# Extract Unique Districts & Save
df_districts = df[['dist_code', 'dist_name', 'state_code']].drop_duplicates()
df_districts.to_csv("districts.csv", index=False)